# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'forest-fires-exp'

experiment=Experiment(ws, experiment_name)

In [3]:
ds = TabularDatasetFactory.from_delimited_files("http://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv")

In [4]:
aml_compute_target = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=aml_compute_target)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)

    aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

Settings and Configurations of AutoML:

* Primary metric: Normalized Mean Absolute Error
  It gives a normalized score of the average difference between the predicted and actual burnt area.
  
* Cross Validation: 5
  The dataset will be split into 5 parts. Four of them will be used for training and one for testing. This is repeated until each part has become the test set once.
  
* Experiment Timeout Minutes: 30

* Max Concurrent Iterations: 4 ( Same as maximum number of nodes in compute cluster )

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=aml_compute_target,
    task='regression',
    primary_metric='normalized_mean_absolute_error',
    training_data = ds,
    label_column_name = 'area',
    n_cross_validations=5,
    **automl_settings
)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
# best_run.get_file_names()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: forest-fires-exp,
Id: AutoML_8222a900-6a71-499c-b2ab-258d4becb642_70,
Type: azureml.scriptrun,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             

In [11]:
#TODO: Save the best model
automl_model = best_run.register_model(model_name = 'best-model-automl',model_path='outputs/model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
env = Environment.get(ws, "AzureML-Tutorial").clone("Inference_Env")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)


deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights=True)
service = Model.deploy(ws, "aciservice", [automl_model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
import requests
import json

scoring_uri = service.scoring_uri

headers = {'Content-Type': 'application/json'}

data = {"data":
        [
          {
            "X":8,
            "Y":6,
            "month":"dec",
            "day":"wed",
            "FFMC":84,
            "DMC":27.8,
            "DC":354.6,
            "ISI":5.3,
            "temp":5.1,
            "RH":61,
            "wind":8,
            "rain":0
          },
          {
            "X":4,
            "Y":6,
            "month":"dec",
            "day":"thu",
            "FFMC":84.6,
            "DMC":26.4,
            "DC":352,
            "ISI":2,
            "temp":5.1,
            "RH":61,
            "wind":4.9,
            "rain":0
            
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Status Code : ", resp.status_code)
print("Predicted Burnt Area : ",resp.text)

Response Status Code :  200
Predicted Burnt Area :  [2.972875914340569, 3.018367635676774]


In [14]:
print(service.get_logs())

2021-02-11T18:20:42,296231421+00:00 - iot-server/run 
2021-02-11T18:20:42,296939426+00:00 - rsyslog/run 
2021-02-11T18:20:42,297309029+00:00 - gunicorn/run 
2021-02-11T18:20:42,305247993+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_f6b93dd0d0e8efaa08690fad952a421b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f6b93dd0d0e8efaa08690fad952a421b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f6b93dd0d0e8efaa08690fad952a421b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f6b93dd0d0e8efaa08690fad952a421b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_f6b93dd0d0e8efaa08690fad952a421b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [15]:
service.delete()

In [16]:
AmlCompute.delete(cpu_cluster)